In [ ]:

import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp

In [ ]:
with open("alldata.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))
print(data.keys())

name_list = data['name_list']
industry_list = data['industry_list']
index_list = data['index_list']
cusip_list = data['cusip_list']
universe = data['universe']

In [ ]:
universe

In [ ]:
rd.open_session()

ric = "ABI.BR"
start_date = "2019-09-01"
end_date = "2025-08-01"

price_df = rd.get_history(
    universe=ric,
    start=start_date,
    end=end_date,
    interval="daily"
)

df = pd.DataFrame({
    'price': price_df['TRDPRC_1'],
    'volume_shares': price_df['ACVOL_UNS'],
    'bid': price_df['BID'],
    'ask': price_df['ASK']
})

fundamental_response = rd.get_data(
    universe=ric,
    fields=[
        "TR.TotalReturn.Date",
        "TR.TotalReturn",           # Total Return Index
        "TR.PriceToBook",           # Price to Book (Market-to-Book)
        "TR.CompanyMarketCap"       # Market Cap
    ],
    parameters={
        "SDate": start_date,
        "EDate": end_date,
        "Frq": "D",
        "Curn": "EUR"
    }
)

print("\nFundamental data response:")
print(fundamental_response.head(10))
print(f"\nColumns: {fundamental_response.columns.tolist()}")

if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
    fund_df = fundamental_response.copy()
    
    if 'Date' in fund_df.columns:
        fund_df['Date'] = pd.to_datetime(fund_df['Date'])
        fund_df.set_index('Date', inplace=True)
        
        if 'Instrument' in fund_df.columns:
            fund_df.drop('Instrument', axis=1, inplace=True)
        
        print(f"\nFundamental data with Date index:")
        print(fund_df.head(10))
        
        df = df.join(fund_df, how='left')
    else:
        print("\nWARNING: No Date column found in fundamental data!")
        print("Available columns:", fund_df.columns.tolist())

column_mapping = {
    'Total Return': 'tri',
    'Price To Book Value': 'mtbv',
    'Company Market Cap': 'cap'
}

for old_name, new_name in column_mapping.items():
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)

df['volume'] = df['volume_shares'] * df['price']
df.drop('volume_shares', axis=1, inplace=True)

if 'mtbv' in df.columns:
    df['mtbv'] = df['mtbv'].ffill()
if 'cap' in df.columns:
    df['cap'] = df['cap'].ffill()

desired_order = ['price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
existing_cols = [col for col in desired_order if col in df.columns]
df = df[existing_cols]

print("\n" + "="*80)
print("FINAL DATASET")
print("="*80)
print(df.head(15))
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nNull counts:\n{df.isnull().sum()}")
print(f"\nSample with non-null data:")
print(df[df['tri'].notna()].head(10))

df.to_csv("ABI_BR_Data.csv", index=True)
print(f"\nData saved to ABI_BR_Data.csv")

rd.close_session()

In [ ]:
import time

rd.open_session()

unique_rics = universe['RIC'].unique()
print(f"Total rows in universe: {len(universe)}")
print(f"Unique RICs to process: {len(unique_rics)}")

start_date = "2019-09-01"
end_date = "2025-08-01"

all_data = []
failed_rics = []

total_rics = len(unique_rics)
print(f"\nProcessing {total_rics} unique stocks...")

for idx, ric in enumerate(unique_rics, 1):
    print(f"[{idx}/{total_rics}] Processing {ric}...", end=' ')
    
    try:
        # --- Fetch historical PRICING data ---
        price_df = rd.get_history(
            universe=ric,
            start=start_date,
            end=end_date,
            interval="daily"
        )
        
        # Extract only what we need
        df = pd.DataFrame({
            'RIC': ric,  # Add RIC column
            'price': price_df['TRDPRC_1'],
            'volume_shares': price_df['ACVOL_UNS'],
            'bid': price_df['BID'],
            'ask': price_df['ASK']
        })
        
        # --- Fetch FUNDAMENTAL data WITH DATES ---
        fundamental_response = rd.get_data(
            universe=ric,
            fields=[
                "TR.TotalReturn.Date",
                "TR.TotalReturn",
                "TR.PriceToBook",
                "TR.CompanyMarketCap"
            ],
            parameters={
                "SDate": start_date,
                "EDate": end_date,
                "Frq": "D",
                "Curn": "EUR"
            }
        )
        
        # Process fundamental data
        if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
            fund_df = fundamental_response.copy()
            
            if 'Date' in fund_df.columns:
                fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                fund_df.set_index('Date', inplace=True)
                
                if 'Instrument' in fund_df.columns:
                    fund_df.drop('Instrument', axis=1, inplace=True)
                
                # Merge with price data
                df = df.join(fund_df, how='left')
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        df['volume'] = df['volume_shares'] * df['price']
        df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill mtbv and cap
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        # Append to list
        all_data.append(df)
        
        print(f"✓ {len(df)} rows")
        
        # Rate limiting - sleep briefly to avoid overwhelming the API
        if idx % 10 == 0:
            time.sleep(2)
        
    except Exception as e:
        print(f"✗ Failed: {e}")
        failed_rics.append(ric)
        continue

# --- 6. Combine all data ---
print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)
print(f"Successfully processed: {len(all_data)} stocks")
print(f"Failed: {len(failed_rics)} stocks")

if failed_rics:
    print(f"\nFailed RICs:")
    for ric in failed_rics:
        print(f"  - {ric}")

# --- 7. Create combined DataFrame ---
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"\nCombined dataset shape: {combined_df.shape}")
    print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"Unique stocks: {combined_df['RIC'].nunique()}")
    
    # Display sample
    print("\nSample data:")
    print(combined_df.head(10))
    
    # Save to CSV
    combined_df.to_csv("all_stocks_data.csv", index=False)
    print(f"\n✓ Saved to all_stocks_data.csv")
    
    # Show null counts
    print("\nNull counts by column:")
    print(combined_df.isnull().sum())
else:
    print("\nNo data retrieved!")

# --- 8. Close session ---
rd.close_session()

print("\nDone!")

In [56]:
pd.read_csv("all_stocks_data.csv").isnull().sum()

RIC            0
Date           0
price       1410
tri          376
volume    332768
cap         3050
bid          499
ask          570
dtype: int64